Business Problem:
Challenge description: As a parent, I've discovered that the is a huge shortage of daycare centers for working parents in the Seattle, WA market. If I were to take the problem into my own hands, I would like to know the optimum neighborhood location to open my own daycare to optimize for the demand for daycare in the city.

Criteria:
1. Placement of the daycare would be close to high concentration of apartment complexes and/or zoned suburban areas so that parents can drop off their children close to home.

2. Daycares should also be further away ideally from venues that are non-kid friendly, such as bars, nightclubs, dispensaries, etc.

Method:
1. Identify all distinct neighborhoods in the Seattle area
2. Query Foursquare to find specific venues in the daycare criteria mentioned in Business Program.
3. Match Seattle neighborhood data to fourquare data that is nearby.
4. Rank neighborhoods against venue critera, showing most common to least common venue in Seattle
3. Select Neighborhood(s) that are the best fit for setting up a new daycare.

Intended Outcome:
Based on analysis of the data, identify best neighborhood location to set up new daycare.

In [1]:
#Importing the usual necessary libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import geopandas as gpd # imported geopandas for shape file import

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Base Libraries imported.')

#Section to import K-Means related libraries
#Initialize libraries
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('K-Means Libraries imported.')


Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Base Libraries imported.
K-Means Libraries imported.


In [2]:
# Load Seattle csv data
seattle_df=pd.read_csv("Seattle Data.csv")
seattle_df

,OBJECTID,CRA_NO,CRA_GRP,Neighborhood,Lat Long,Latitude,Longitude,DETL_NAMES,District,AREA_ACRES,AREA_SQMI,SHAPE_Length,SHAPE_Area,Population,Population_in_HH,Households,Family_HHs,Avg_HH_Size,Pop_Under18
0,1,1.1,1,Arbor Heights,"47.51, -122.38",47.508980,-122.379693,"Arbor Heights, Brace Point, Endolyne, Arroyo",Southwest,785.9,1.23,27702.72765,3.423387e+07,6278,6260,2702,1773,2.32,1287
1,2,1.2,1,Fauntleroy,"47.52, -122.39",47.524880,-122.386935,"Fauntleroy, Gatewood, Morgan Junction, Lincoln...",Southwest,1393.1,2.18,48830.04832,6.068510e+07,15227,15147,6823,3944,2.22,2814
2,3,1.3,1,West Seattle Junction,"47.55, -122.38",47.563121,-122.386722,"Seaview, Fairmount Park, Morgan Junction, Gene...",Southwest,1447.5,2.26,38739.89626,6.305462e+07,20345,20063,9448,4966,2.12,3716
3,4,1.4,1,Alki,"47.57, -122.41",47.580508,-122.386705,"Alki, Admiral",Southwest,898.3,1.40,50339.65642,3.912974e+07,12037,11882,5888,3186,2.02,1946
4,5,2.1,2,North Delridge,"47.56, -122.36",47.563673,-122.364989,"North Delridge, Pigeon Point, Avalon, Luna Par...",Delridge Neighborhoods,1606.9,2.51,64237.74268,6.999494e+07,5675,5658,2858,1412,1.98,818
5,6,2.2,2,High Point,"47.55, -122.37",47.545718,-122.368680,"High Point, Delridge",Delridge Neighborhoods,553.2,0.86,22626.00822,2.409669e+07,8336,8281,3025,1920,2.74,2495
6,7,2.3,2,Riverview,"47.54, -122.35",47.548226,-122.354897,"Riverview, Delridge",Delridge Neighborhoods,1005.9,1.57,37828.42176,4.381726e+07,4970,4941,1861,1210,2.66,1074
7,8,2.4,2,Roxhill,"47.53, -122.37",47.528392,-122.371318,"Roxhill, Westwood, South Delridge, White Center",Delridge Neighborhoods,978.5,1.53,33451.49863,4.262265e+07,13045,13045,5715,3174,2.28,2485
8,9,2.5,2,Highland Park,"47.53, -122.35",47.527299,-122.349214,"Highland Park, Westwood, White Center, South D...",Delridge Neighborhoods,897.7,1.40,31097.84366,3.910323e+07,7054,7029,3012,1553,2.33,1555
9,10,3.1,3,South Park,"47.53, -122.33",47.529862,-122.324658,South Park,Greater Duwamish,693.9,1.08,36300.10317,3.022804e+07,3719,3627,1413,735,2.57,1094


In [3]:
# display area around Seattle and add neighborhood markers

seattle_map = folium.Map(location=[47.608013, -122.335167], zoom_start=11.4)

# add markers to map
for lat, lng, Neighborhood, District in zip(seattle_df['Latitude'], seattle_df['Longitude'], seattle_df['Neighborhood'], seattle_df['District']):
    label = '{}, {}'.format(Neighborhood, District)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(seattle_map)  
    

seattle_map

In [4]:
# Repeat 4 square exercise for Seattle, get max of 500 sites
CLIENT_ID = 'TYPO2YBPRE2LLGSJDIKKGKTOPTG4IPJQINEDJTUUOBVTETTT' # your Foursquare ID
CLIENT_SECRET = '2Z4GD2X4F30DCHRCJEW2OFH3GXQDENGE4MGH52WWGJOICEQW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#Seattle Lat and Long
neighborhood_latitude= 47.608013 
neighborhood_longitude=-122.335167


LIMIT = 500 # limit of number of venues returned by Foursquare API

radius = 600 # define search radius, using 600 meters

Categoryids="5744ccdfe4b0c0459246b4c7,4bf58dd8d48988d116941735,4e67e38e036454776db1fb3a,#4d4b7105d754a06376d81259,#52c71aaf3cf9994f4e043d17"

#Categories selected:
#5744ccdfe4b0c0459246b4c7 - Childcare service
#4bf58dd8d48988d116941735 - Bars
#4e67e38e036454776db1fb3a - Residences
#4d4b7105d754a06376d81259 - Nighlife Spot
#52c71aaf3cf9994f4e043d17 - Marijuana Dispensary

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT,
    Categoryids)
#url # display URL
results = requests.get(url).json()

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Taproom at Pike Place,Bar,47.609681,-122.340746
1,Elysian Bar,Bar,47.609699,-122.339334
2,Forge Lounge,Bar,47.603185,-122.337254
3,Sonya's Skyy Lounge,Bar,47.610693,-122.341851
4,Elephant & Castle,Bar,47.610186,-122.334925


In [5]:
nearby_venues.shape

(93, 4)

In [6]:
# Create function to get nearby venues, focusing on the 3 types of venues that were noted as criteria
def getNearbyVenues(names, latitudes, longitudes, radius=radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT,
            Categoryids)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [7]:
# Use newly created function of nearby venues on seattle data

seattle_venues = getNearbyVenues(names=seattle_df['Neighborhood'],
                                   latitudes=seattle_df['Latitude'],
                                   longitudes=seattle_df['Longitude']
                                  )


Arbor Heights
Fauntleroy
West Seattle Junction
Alki
North Delridge
High Point
Riverview
Roxhill
Highland Park
South Park
Georgetown
SODO
South Beacon Hill
Beacon Hill
Rainier Beach
Columbia City
Seward Park
Mt. Baker
Madrona
Judkins Park
First Hill
Capitol Hill
Madison Park
Montlake
University District
Ravenna
Wedgwood
Laurelhurst
Northgate
Olympic Hills
Cedar Park
Broadview
Licton Springs
Greenwood
Green Lake
Pinehurst
North Beach
Whittier Heights
Sunset Hill
Ballard
Fremont
Wallingford
Eastlake
Magnolia
Interbay
Queen Anne
Belltown
Downtown Commercial Core
International District


In [8]:
# check venues dataframe to make sure all data came over properly
print(seattle_venues.shape)
seattle_venues.head()

(600, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arbor Heights,47.508980,-122.379693,Arbor Heights Elder Care,47.505092,-122.375754,Assisted Living
1,Fauntleroy,47.524880,-122.386935,SYNERGY HomeCare,47.521952,-122.387882,Home Service
2,West Seattle Junction,47.563121,-122.386722,West 5,47.561760,-122.386930,Bar
3,West Seattle Junction,47.563121,-122.386722,Elliott Bay Brewery and Pub,47.560418,-122.386602,Brewery
4,West Seattle Junction,47.563121,-122.386722,Shadowland,47.562952,-122.386653,Bar


In [9]:
# find unique categories within dataset
print('There are {} unique categories.'.format(len(seattle_venues['Venue Category'].unique())))

There are 68 unique categories.


In [10]:
# Do one hot encoding on the dataset
seattle_onehot = pd.get_dummies(seattle_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
seattle_onehot['Neighborhood'] = seattle_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [seattle_onehot.columns[-1]] + list(seattle_onehot.columns[:-1])
seattle_onehot = seattle_onehot[fixed_columns]

seattle_onehot.head()
#seattle_onehot.shape

,Wine Bar,American Restaurant,Arcade,Assisted Living,Bakery,Bar,Beer Bar,Beer Garden,Beer Store,Boat or Ferry,...,Rock Club,Sake Bar,School,Seafood Restaurant,Speakeasy,Sports Bar,Sushi Restaurant,Travel Lounge,Vegetarian / Vegan Restaurant,Whisky Bar
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
seattle_grouped = seattle_onehot.groupby('Neighborhood').mean().reset_index()
seattle_grouped

,Neighborhood,Wine Bar,American Restaurant,Arcade,Assisted Living,Bakery,Bar,Beer Bar,Beer Garden,Beer Store,...,Rock Club,Sake Bar,School,Seafood Restaurant,Speakeasy,Sports Bar,Sushi Restaurant,Travel Lounge,Vegetarian / Vegan Restaurant,Whisky Bar
0,Alki,0.111111,0.000000,0.000000,0.222222,0.00,0.222222,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Arbor Heights,0.000000,0.000000,0.000000,1.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Ballard,0.000000,0.000000,0.000000,0.027027,0.00,0.297297,0.000000,0.027027,0.000000,...,0.027027,0.000000,0.000000,0.027027,0.000000,0.000000,0.054054,0.000000,0.027027,0.000000
3,Belltown,0.020000,0.000000,0.000000,0.000000,0.00,0.230000,0.010000,0.000000,0.000000,...,0.010000,0.000000,0.000000,0.020000,0.020000,0.010000,0.010000,0.000000,0.000000,0.000000
4,Broadview,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.250000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Capitol Hill,0.000000,0.000000,0.000000,0.000000,0.00,0.272727,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000
6,Cedar Park,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Columbia City,0.000000,0.000000,0.000000,0.000000,0.00,0.222222,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.111111
8,Downtown Commercial Core,0.021978,0.010989,0.000000,0.021978,0.00,0.263736,0.021978,0.000000,0.000000,...,0.000000,0.010989,0.000000,0.010989,0.010989,0.021978,0.010989,0.010989,0.000000,0.000000
9,Eastlake,0.000000,0.000000,0.000000,0.000000,0.00,0.300000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [12]:
# create function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [13]:
# Take a look at the top 5 common venues in the neighborhood
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = seattle_grouped['Neighborhood']

for ind in np.arange(seattle_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(seattle_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Alki,Assisted Living,Pub,Bar,Wine Bar,Cocktail Bar
1,Arbor Heights,Assisted Living,Whisky Bar,Deli / Bodega,Cocktail Bar,Coffee Shop
2,Ballard,Bar,Residential Building (Apartment / Condo),Pub,Cocktail Bar,Sushi Restaurant
3,Belltown,Residential Building (Apartment / Condo),Bar,Cocktail Bar,Building,Dive Bar
4,Broadview,Residential Building (Apartment / Condo),Beer Bar,Whisky Bar,Daycare,Cocktail Bar


In [14]:
# Filter by neighborhoods with most residentials buildings
residential_df=neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['1st Most Common Venue']== 'Residential Building (Apartment / Condo)']
residential_rows=residential_df.shape[0]
print('Number of neighborhoods with highest residential venues  :',residential_rows)

Number of neighborhoods with highest residential venues  : 12


In [15]:
# See the residential neighborhood names
residential_df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Belltown,Residential Building (Apartment / Condo),Bar,Cocktail Bar,Building,Dive Bar
4,Broadview,Residential Building (Apartment / Condo),Beer Bar,Whisky Bar,Daycare,Cocktail Bar
5,Capitol Hill,Residential Building (Apartment / Condo),Bar,Cocktail Bar,Pub,Speakeasy
7,Columbia City,Residential Building (Apartment / Condo),Bar,Whisky Bar,Sports Bar,Pub
11,First Hill,Residential Building (Apartment / Condo),Bar,Cocktail Bar,Building,Pizza Place
19,International District,Residential Building (Apartment / Condo),Bar,Sports Bar,Building,Karaoke Bar
20,Judkins Park,Residential Building (Apartment / Condo),Whisky Bar,Daycare,Cocktail Bar,Coffee Shop
27,North Delridge,Residential Building (Apartment / Condo),Beer Garden,Whisky Bar,Daycare,Cocktail Bar
28,Northgate,Residential Building (Apartment / Condo),Assisted Living,Preschool,Whisky Bar,Creperie
29,Olympic Hills,Residential Building (Apartment / Condo),Housing Development,Whisky Bar,Daycare,Cocktail Bar


In [16]:
# Filter next by neighborhoods least number of alchoholic (bar pubs) establishment
alchoholic=['Wine Bar','Whisky Bar','Bar','Pub','Beer Bar','Dive Bar','Beer Garden','Karaoke Bar']
residential_df2=residential_df.loc[~residential_df['2nd Most Common Venue'].isin(alchoholic)]
residential_rows2=residential_df2.shape[0]
print('Number of residential neighborhood, least number of bars :',residential_rows2)

Number of residential neighborhood, least number of bars : 2


In [17]:
#Lets take a look at these remaining neighborhoods
residential_df2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
28,Northgate,Residential Building (Apartment / Condo),Assisted Living,Preschool,Whisky Bar,Creperie
29,Olympic Hills,Residential Building (Apartment / Condo),Housing Development,Whisky Bar,Daycare,Cocktail Bar


In [18]:
# Now we have shortlist of neighborhoods, lets look at these neighborhoods' populations
selected_n=residential_df2['Neighborhood'].values.tolist()
selected_hoods=seattle_df.loc[seattle_df['Neighborhood'].isin(selected_n)]

shortlist_df=selected_hoods.filter(items=['CRA_NO','Neighborhood','Latitude','Longitude','Population','Population_in_HH','Households','Family_HHs','Avg_HH_Size','Pop_Under18'])
#selected_hoods

In [19]:
# Sort by Highest Population, and with highest number of family households
rankedlist_df=shortlist_df.sort_values(['Population_in_HH', 'Family_HHs'], ascending=[False, False])
rankedlist_df

,CRA_NO,Neighborhood,Latitude,Longitude,Population,Population_in_HH,Households,Family_HHs,Avg_HH_Size,Pop_Under18
29,8.2,Olympic Hills,47.727349,-122.303088,15618,15501,6968,3831,2.22,2450
28,8.1,Northgate,47.712290,-122.325621,15615,15377,7713,3145,1.99,2361


In [20]:
#Based on Venue data combined with demographic data, show best neighborhood!
neighborhood_df=rankedlist_df.iloc[0]
Latitude=neighborhood_df['Latitude']
Longitude=neighborhood_df['Longitude']
Neighborhood=neighborhood_df['Neighborhood']
print('The best neighborhood for a daycare is',Neighborhood)

The best neighborhood for a daycare is Olympic Hills


In [21]:
# Going back to Foursquare data, now lets locate key venues and their locations in our selected neighborhood

CLIENT_ID = 'TYPO2YBPRE2LLGSJDIKKGKTOPTG4IPJQINEDJTUUOBVTETTT' # your Foursquare ID
CLIENT_SECRET = '2Z4GD2X4F30DCHRCJEW2OFH3GXQDENGE4MGH52WWGJOICEQW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 500 # limit of number of venues returned by Foursquare API

radius = 1000 # define search radius, using 1000 meters

Categoryids="5744ccdfe4b0c0459246b4c7,4bf58dd8d48988d116941735,4e67e38e036454776db1fb3a,#4d4b7105d754a06376d81259,#52c71aaf3cf9994f4e043d17"

#Categories selected:
#5744ccdfe4b0c0459246b4c7 - Childcare service
#4bf58dd8d48988d116941735 - Bars
#4e67e38e036454776db1fb3a - Residences
#4d4b7105d754a06376d81259 - Nighlife Spot
#52c71aaf3cf9994f4e043d17 - Marijuana Dispensary

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Latitude, 
    Longitude, 
    radius, 
    LIMIT,
    Categoryids)
#url # display URL
results = requests.get(url).json()

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
target_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
target_venues =target_venues.loc[:, filtered_columns]

# filter the category for each row
target_venues['venue.categories'] = target_venues.apply(get_category_type, axis=1)

# clean columns
target_venues.columns = [col.split(".")[-1] for col in target_venues.columns]

target_venues



,name,categories,lat,lng
0,Elliott Bay Public House & Brewery,Pub,47.720373,-122.295006
1,Caroline Tavern,Bar,47.728623,-122.312436
2,Brother Barrel,Beer Bar,47.720289,-122.294972
3,Executive Estates,Residential Building (Apartment / Condo),47.731461,-122.312147
4,Promenade Apartments,Residential Building (Apartment / Condo),47.731609,-122.293633
5,Lake City Bar and Grill,Pub,47.721735,-122.293913
6,Solara Apartments - Building C,Residential Building (Apartment / Condo),47.721881,-122.293442
7,3030 Lake City,Building,47.732244,-122.294999
8,The Beer Authority,Beer Store,47.721910,-122.293479


In [22]:
# Split out residential buildings and everything else
housing=['Neighborhood','Assisted Living','Residential Building (Apartment / Condo)']
housing_venues=target_venues.loc[target_venues['categories'].isin(housing)]
#housing_venues

#Other_venues=target_venues.loc[target_venues['categories']!='Residential Building (Apartment / Condo)']
other_venues=target_venues[np.logical_not(target_venues['categories'].isin(housing))]
# df[np.logical_not(df["column"].isin(["value"]))]

#other_venues

In [23]:
# display area around selected neighborhood, color code green for residental building, blue for others.

neighborhood_map = folium.Map(location=[Latitude, Longitude], zoom_start=15)

#section for residential buildings
for lat, lng, categories, name in zip(housing_venues['lat'], housing_venues['lng'], housing_venues['categories'], housing_venues['name']):
    label = '{}, {}'.format(categories, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='darkgreen',
        fill_opacity=0.7,
        parse_html=False).add_to(neighborhood_map)   

folium.Marker([Latitude,Longitude],popup=Neighborhood).add_to(neighborhood_map)   

for lat, lng, categories, name in zip(other_venues['lat'], other_venues['lng'], other_venues['categories'], other_venues['name']):
    label = '{}, {}'.format(categories, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(neighborhood_map)   

folium.Marker([Latitude,Longitude],popup=Neighborhood).add_to(neighborhood_map)   


    
neighborhood_map